# YOUTUBE SENTIMENT ANALYSIS

## IF 4072 - Pemrosesan Bahasa Alami

* Jason Wiguna - 13516024 
* Kevin Basuki - 13516071
* Nicolaus Boby - 13516077
* Steven Sukma L - 13516102
* Eric Jonathan - 13516117

In [1]:
from emoji import UNICODE_EMOJI, emoji_lis
import emojis
import pandas as pd
import numpy as np
import math

In [2]:
# Kralj Novak, Petra; Smailović, Jasmina; Sluban, Borut and Mozetič, Igor, 2015, Emoji Sentiment Ranking 1.0, Slovenian language resource repository CLARIN.SI, http://hdl.handle.net/11356/1048.
emo_df = pd.read_csv("Data/Emoji_Sentiment_Data_v1.0.csv")
emo_df.head()

,Emoji,Unicode codepoint,Occurrences,Position,Negative,Neutral,Positive,Unicode name,Unicode block
0,😂,0x1f602,14622,0.805101,3614,4163,6845,FACE WITH TEARS OF JOY,Emoticons
1,❤,0x2764,8050,0.746943,355,1334,6361,HEAVY BLACK HEART,Dingbats
2,♥,0x2665,7144,0.753806,252,1942,4950,BLACK HEART SUIT,Miscellaneous Symbols
3,😍,0x1f60d,6359,0.765292,329,1390,4640,SMILING FACE WITH HEART-SHAPED EYES,Emoticons
4,😭,0x1f62d,5526,0.803352,2412,1218,1896,LOUDLY CRYING FACE,Emoticons


In [3]:
emo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969 entries, 0 to 968
Data columns (total 9 columns):
Emoji                969 non-null object
Unicode codepoint    969 non-null object
Occurrences          969 non-null int64
Position             969 non-null float64
Negative             969 non-null int64
Neutral              969 non-null int64
Positive             969 non-null int64
Unicode name         969 non-null object
Unicode block        969 non-null object
dtypes: float64(1), int64(4), object(4)
memory usage: 68.2+ KB


In [4]:
emo_df['score'] = (emo_df['Positive'] - emo_df['Negative']) / (emo_df['Negative'] + emo_df['Positive'] + emo_df['Neutral'])

In [5]:
def get_emoji_sentiments(df, text):
    emos = []
    raw = emoji_lis(text)
    for item in raw:
        emos.append(list(item.values()))
    total = 0
    for emo in emos:
        total += float(df.loc[df['Emoji'] == emo[1]]['score'])
    return np.tanh(total)

In [6]:
data = pd.read_csv('Data/labeled.csv', names=['Comments', 'Label'])
data.head()

,Comments,Label
0,Hnya mungkin iman nya masih goyah dan alasan t...,-0.561180
1,"Kok pada ngributin hidup orang lain, mau pinda...",0.012860
2,Isi suratnya adalah ....\r\nRina nose tidak pe...,0.000000
3,"hmmmm sayang nya rina kalau ampe pindah,ibarat...",0.014891
4,👍👍👍👍,0.975000


In [11]:
comments = data['Comments']
# for comment in comments:
#     print(get_emoji_sentiments(emo_df, comment))

In [12]:
from string import punctuation
import nltk
def remove_punctuation(s):
    s = ''.join([i for i in s if i not in frozenset(punctuation)])
    return s

data['cleaned'] = data['Comments'].apply(remove_punctuation)

def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

data['tokenized_sents'] = data.apply(lambda row: nltk.word_tokenize(row['cleaned']), axis=1)

print(data['tokenized_sents'])

0       [Hnya, mungkin, iman, nya, masih, goyah, dan, ...
1       [Kok, pada, ngributin, hidup, orang, lain, mau...
2       [Isi, suratnya, adalah, Rina, nose, tidak, per...
3       [hmmmm, sayang, nya, rina, kalau, ampe, pindah...
4                                                  [👍👍👍👍]
5                                         [Azka, ganteng]
6       [Yes, om, Deddy, you, have, built, the, monste...
7       [dapat, suami, bule, dia, jadi, pindah, agama,...
8           [apa, yang, di, madsud, air, kejujuran, rina]
9                                [Yg, nonton, 2019, like]
10      [Bang, sebenar, nya, apa, isi, surat, rina, bang]
11                      [Salam, dari, palangkaraya, bang]
12                            [Gaya, mu, itu, lohhhwkwkw]
13           [Jd, bingung, apa, isi, surat, rina, nose😱😲]
14      [Klo, ngedidik, anak, jjr, gwe, sama, dgn, Om,...
15                             [Om, kapan, punya, rambut]
16                               [Isi, surat, mulai, 630]
17            

In [ ]:
# Word Normalization
import re
import collections as col

def words(text): return re.findall(r'\w+', text.lower())

WORDS = col.Counter(words(open('kata.txt').read()))

def P(word, N=sum(WORDS.values())):
    # "Probability of `word`."
    # print(N)
    return WORDS[word] / N

def correction(word):
    # "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word):
    # "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    # "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    # "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)] # [('', 'kemarin'), ('k', 'emarin'), ('ke', 'marin'), dst]
    deletes    = [L + R[1:]               for L, R in splits if R] # ['emarin', 'kmarin', 'kearin', dst]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1] # ['ekmarin', 'kmearin', 'keamrin', dst]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters] # ['aemarin', 'bemarin', 'cemarin', dst]
    inserts    = [L + c + R               for L, R in splits for c in letters] # ['akemarin', 'bkemarin', 'ckemarin', dst]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    # "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

for i,row in enumerate(data['tokenized_sents']):
    for j,kata in enumerate(row):
        data['tokenized_sents'][i][j] = correction(kata)

print(data['tokenized_sents'])

In [ ]:
import nltk
import pycrfsuite
# nltk.download()

ct = nltk.tag.CRFTagger()

In [14]:
ct.set_model_file('all_indo_man_tag_corpus_model.crf.tagger')
hasil = ct.tag_sents(data['tokenized_sents'])
hasil

[[('nya', 'PRP'),
  ('mungkin', 'MD'),
  ('iman', 'VB'),
  ('nya', 'PRP'),
  ('masih', 'MD'),
  ('goyah', 'VB'),
  ('dan', 'CC'),
  ('alasan', 'NN'),
  ('tertentu', 'PR'),
  ('dgn', 'RB'),
  ('melepas', 'VB'),
  ('hijau', 'NN'),
  ('loak', 'NN'),
  ('semoga', 'NN'),
  ('cina', 'SC'),
  ('kembali', 'VB'),
  ('mendapatkan', 'VB'),
  ('kebaikan', 'VB'),
  ('lagi', 'RB'),
  ('karena', 'SC'),
  ('dia', 'PRP'),
  ('sedang', 'MD'),
  ('di', 'IN'),
  ('uji', 'NN')],
 [('kok', 'VB'),
  ('pada', 'IN'),
  ('ngributin', 'NN'),
  ('hidup', 'NN'),
  ('orang', 'NN'),
  ('lain', 'JJ'),
  ('mau', 'RB'),
  ('pindah', 'VB'),
  ('agama', 'NN'),
  ('kek', 'NN'),
  ('mau', 'RB'),
  ('buka', 'VB'),
  ('hijau', 'NN'),
  ('kek', 'NN'),
  ('mau', 'RB'),
  ('ngapain', 'JJ'),
  ('saja', 'RB'),
  ('kek', 'NN'),
  ('itu', 'PR'),
  ('kan', 'RP'),
  ('hidup', 'NN'),
  ('mereka', 'PRP'),
  ('kok', 'VB'),
  ('kian', 'NN'),
  ('yg', 'NN'),
  ('kecoh', 'NN'),
  ('toh', 'NN'),
  ('makan', 'VB'),
  ('pun', 'RP'),
  ('nggak